# データ ドリフトの監視

時間が経つにつれて、フィーチャ データの傾向の変化により、モデルの正確な予測の効果が低下する可能性があります。この現象は*データ ドリフト*と呼ばれ、必要に応じてモデルを再トレーニングできるように、機械学習ソリューションを監視して検出することが重要です。

このラボでは、データセットのデータ ドリフト監視を構成します。

## DataDriftDetector モジュールをインストールする

データ ドリフト モニターを定義するには、最新バージョンの Azure ML SDK がインストールされていることを確認し、**datadrift** モジュールをインストールして、以下の通り後続のセルを実行します。

In [ ]:
!pip install --upgrade azureml-sdk[notebooks,automl,explain]
!pip install --upgrade azureml-datadrift
# インストールが完了したら、カーネルを再起動してください。

> **重要**: <u>カーネルを再起動</u>する必要があります。Jupyter の **[カーネル]** メニューで、**[再起動と出力のクリア]** を選択します。次に、上のセルからの出力が削除され、カーネルが再起動されたら、以下の手順を続行します。

## ワークスペースに接続する

これで、Azure ML SDK を使用してワークスペースに接続する準備が整いました。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## ベースライン データセットを作成する

データ ドリフトのデータセットを監視するには、*ベースライン* データセット (通常、モデルのトレーニングに使用されるデータセット) を登録して、将来収集されるデータとの比較ポイントとして使用する必要があります。 

In [ ]:
from azureml.core import Datastore, Dataset


# ベースライン データをアップロードする
default_ds = ws.get_default_datastore()
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'],
                       target_path='diabetes-baseline',
                       overwrite=True, 
                       show_progress=True)

# ベースライン データセットを作成して登録する
print('Registering baseline dataset...')
baseline_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-baseline/*.csv'))
baseline_data_set = baseline_data_set.register(workspace=ws, 
                           name='diabetes baseline',
                           description='diabetes baseline data',
                           tags = {'format':'CSV'},
                           create_new_version=True)

print('Baseline dataset registered!')

## ターゲット データセットを作成する

時間の経過とともに、ベースライン トレーニング データと同じ機能を持つ新しいデータを収集できます。この新しいデータをベースライン データと比較するには、データ ドリフトを分析する機能を含むターゲット データセットと、新しいデータが最新であった時点を示すタイムスタンプ フィールドを定義する必要があります。これにより、一時的なサイクル間隔でのデータ ドリフトを測定します。タイムスタンプは、データセット自体のフィールド、またはデータの格納に使用されるフォルダーとファイル名パターンから派生したフィールドのいずれかです。たとえば、月のフォルダーを含む年のフォルダーと、その日のフォルダーを含むフォルダー階層に新しいデータを保存できます。または、次のようにファイル名に年、月、日をエンコードすることもできます。*data_2020-01-29.csv*。これは、次のコードで採用されているアプローチです

In [ ]:
import datetime as dt
import pandas as pd

print('Generating simulated data...')

# 2 つのデータ ファイルのうち小さい方を読み込む
data = pd.read_csv('data/diabetes2.csv')

# 過去 6 週間のデータを生成します
weeknos = reversed(range(6))

file_paths = []
for weekno in weeknos:
    
    # X 週間前の日付を取得する
    data_date = dt.date.today() - dt.timedelta(weeks=weekno)
    
    # データを変更してドリフトを作成する
    data['Pregnancies'] = data['Pregnancies'] + 1
    data['Age'] = round(data['Age'] * 1.2).astype(int)
    data['BMI'] = data['BMI'] * 1.1
    
    # ファイル名にエンコードされた日付でファイルを保存する
    file_path = 'data/diabetes_{}.csv'.format(data_date.strftime("%Y-%m-%d"))
    data.to_csv(file_path)
    file_paths.append(file_path)

# ファイルをアップロードする
path_on_datastore = 'diabetes-target'
default_ds.upload_files(files=file_paths,
                       target_path=path_on_datastore,
                       overwrite=True,
                       show_progress=True)

# フォルダー パーティション形式を使用して、タイムスタンプ列 'date' を持つデータセットを定義する
partition_format = path_on_datastore + '/diabetes_{date:yyyy-MM-dd}.csv'
target_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, path_on_datastore + '/*.csv'),
                                                       partition_format=partition_format)

# ターゲット データセットを登録する
print('Registering target dataset...')
target_data_set = target_data_set.with_timestamp_columns('date').register(workspace=ws,
                                                                          name='diabetes target',
                                                                          description='diabetes target data',
                                                                          tags = {'format':'CSV'},
                                                                          create_new_version=True)

print('Target dataset registered!')

## データ ドリフト モニターを作成する

これで、糖尿病データのデータ ドリフト モニターを作成する準備が整いました。データ ドリフト モニターは、定期的またはオンデマンドで実行され、ベースライン データセットとターゲット データセットを比較し、時間の経過とともに新しいデータが追加されます。

### コンピューティング先を作成する

データ ドリフト モニターを実行するには、コンピューティング先が必要です。このラボでは、前に作成したコンピューティング クラスターを使用します (存在しない場合は作成されます)。

> **重要**: 実行する前に、*your-compute-cluster* を以下のコードのコンピューティング クラスターの名前に変更してください。クラスター名は、長さが 2 〜 16 文字のグローバルに一意の名前である必要があります。文字、数字、および - が有効です。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-compute-cluster"

try:
    # 既存のコンピューティング先を確認する
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # まだ存在しない場合は、作成します
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

### データ ドリフト モニターを定義する

これで、**DataDriftDetector** クラスを使用してデータのデータ ドリフト モニターを定義する準備ができました。データ ドリフトを監視する機能、監視プロセスの実行に使用するコンピューティング先の名前、データの比較頻度、アラートがトリガーされるデータのドリフトしきい値、データ収集を可能にする待機時間 (時間単位)を指定できます。

In [ ]:
from azureml.datadrift import DataDriftDetector

# フィーチャーリストを設定する
features = ['Pregnancies', 'Age', 'BMI']

# データ ドリフト検出機能を設定する
monitor = DataDriftDetector.create_from_datasets(ws, 'diabetes-drift-detector', baseline_data_set, target_data_set,
                                                      compute_target=cluster_name, 
                                                      frequency='Week', 
                                                      feature_list=features, 
                                                      drift_threshold=.3, 
                                                      latency=24)
monitor

## モニターをバックフィルする

6 週間のシミュレートされた毎週のデータ収集を含むベースライン データセットとターゲット データセットがあります。これを使用してモニターをバックフィルして、元のベースラインとターゲット データの間のデータ ドリフトを分析できます。

> **注** バックフィル分析を実行するには、コンピューティング先を起動する必要があるため、実行に時間がかかる場合があります。ウィジェットは常に更新されて状態が表示されない場合があるため、リンクをクリックして、Azure Machine Learning Studio で実験の状態を確認してください。

In [ ]:
from azureml.widgets import RunDetails

backfill = monitor.backfill( dt.datetime.now() - dt.timedelta(weeks=6), dt.datetime.now())

RunDetails(backfill).show()
backfill.wait_for_completion()

## データ ドリフトを分析する

次のコードを使用して、バックフィル実行で収集された時点のデータ ドリフトを調べることができます。

In [ ]:
drift_metrics = backfill.get_metrics()
for metric in drift_metrics:
    print(metric, drift_metrics[metric])

次の手順に従って、[Azure Machine Learning Studio](https://ml.azure.com) でデータのドリフト メトリックを視覚化することもできます。

1.**データセット** ページで、**データセット モニター** タブを表示します。
2.表示するデータ ドリフト モニターをクリックします。
3.データ ドリフト メトリックを表示する日付範囲を選択します (列グラフに複数の週のデータが表示されない場合は、1 分ほど待ってから**更新**をクリックします)。
4.上部の**ドリフト概要**セクションのグラフを調べ、全体的なドリフトの大きさとフィーチャごとのドリフトの寄与度を表示します。
5.下部の**機能の詳細**セクションのグラフを確認すると、個々のフィーチャのさまざまなドリフトの尺度を確認できます。

> **注**: データのドリフト メトリックの理解については、Azure Machine Learning のドキュメントの「[データセットを監視する方法](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets#データ ドリフトの結果を理解する)」を参照してください。

## さらに詳しく見る

このラボは、データ ドリフト監視の概念と原則を紹介することを目的としています。データセットを使用したデータ ドリフトの監視の詳細については、Azure Machine Learning のドキュメントの[データセット上のデータ ドリフトの検出](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-datasets)を参照してください。

Azure Kubernetes Service (AKS) クラスターにデプロイされたサービスのデータ ドリフト監視を構成することもできます。この詳細については、Azure Machine Learning ドキュメントの「[Azure Kubernetes Service (AKS) にデプロイされたモデルでデータ ドリフトを検出する](https://docs.microsoft.com/azure/machine-learning/how-to-monitor-data-drift)」を参照してください。
